In [64]:
import sqlalchemy
import pandas as pd
# from sqlalchemy.orm import sessionmaker
import requests
import json
from datetime import datetime
import datetime
import sqlite3

In [65]:
DATABASE_LOCATION = "sqlite:///my_played_tracks.sqlite"
USER_ID = "a8aw76qyjncw3vttuix7mlpg3" # your Spotify username 
TOKEN = "BQBxTuxMXuRnHEv12Bece_IZ0C6DHEuYug6ROzYrLb1zWmbJMqWa_AetHBkoogeMw7Rjwdvy6D0R6aICZNaltGAuCl-t6VQwdRLcloDW2jpOsFjsNEAEgBk38JCMvxRXpmryJfVe24CuiYYLJOFEEqaNpAUUDwN_yWPv"

# Generate your token here:  https://developer.spotify.com/console/get-recently-played/
# Note: You need a Spotify account (can be easily created for free)

def check_if_valid_data(df: pd.DataFrame) -> bool:
    # Check if dataframe is empty
    if df.empty:
        print("No songs downloaded. Finishing execution")
        return False 

    # Primary Key Check
    if pd.Series(df['played_at']).is_unique:
        pass
    else:
        raise Exception("Primary Key check is violated")

    # Check for nulls
    if df.isnull().values.any():
        raise Exception("Null values found")

    # Check that all timestamps are of yesterday's date
#     yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
#     yesterday = yesterday.replace(hour=0, minute=0, second=0, microsecond=0)

#     timestamps = df["timestamp"].tolist()
#     for timestamp in timestamps:
#         if datetime.datetime.strptime(timestamp, '%Y-%m-%d') != yesterday:
#             raise Exception("At least one of the returned songs does not have a yesterday's timestamp")

    return True

if __name__ == "__main__":

    # Extract part of the ETL process
 
    headers = {
        "Accept" : "application/json",
        "Content-Type" : "application/json",
        "Authorization" : "Bearer {token}".format(token=TOKEN)
    }
    
    # Convert time to Unix timestamp in miliseconds      
    today = datetime.datetime.now()
    yesterday = today - datetime.timedelta(days=1)
    yesterday_unix_timestamp = int(yesterday.timestamp()) * 1000

    # Download all songs you've listened to "after yesterday", which means in the last 24 hours      
    r = requests.get("https://api.spotify.com/v1/me/player/recently-played?after={time}".format(time=yesterday_unix_timestamp), headers = headers)

    data = r.json()
    
    song_names = []
    artist_names = []
    played_at_list = []
    timestamps = []

    # Extracting only the relevant bits of data from the json object      
    for song in data["items"]:
        song_names.append(song["track"]["name"])
        artist_names.append(song["track"]["album"]["artists"][0]["name"])
        played_at_list.append(song["played_at"])
        timestamps.append(song["played_at"][0:10])
        
    # Prepare a dictionary in order to turn it into a pandas dataframe below       
    song_dict = {
        "song_name" : song_names,
        "artist_name": artist_names,
        "played_at" : played_at_list,
        "timestamp" : timestamps
    }

    song_df = pd.DataFrame(song_dict, columns = ["song_name", "artist_name", "played_at", "timestamp"])
    
    # Validate
    if check_if_valid_data(song_df):
        print("Data valid, proceed to Load stage")

    # Load

    engine = sqlalchemy.create_engine(DATABASE_LOCATION)
    conn = sqlite3.connect('my_played_tracks.sqlite')
    cursor = conn.cursor()

    sql_query = """
    CREATE TABLE IF NOT EXISTS my_played_tracks(
        song_name VARCHAR(200),
        artist_name VARCHAR(200),
        played_at VARCHAR(200),
        timestamp VARCHAR(200),
        CONSTRAINT primary_key_constraint PRIMARY KEY (played_at)
    )
    """

    cursor.execute(sql_query)
    print("Opened database successfully")

    try:
        song_df.to_sql("my_played_tracks", con = engine, index=False, if_exists='replace')
    except:
        print("Data already exists in the database")

    conn.close()
    print("Close database successfully")
    

Data valid, proceed to Load stage
Opened database successfully
Close database successfully


In [66]:
song_names

['Lakeeran (From "Haseen Dillruba")',
 'Tu Hi Yaar Mera (From "Pati Patni Aur Woh")',
 'Illegal Weapon 2.0 (From "Street Dancer 3D")',
 'Lagdi Lahore Di (From "Street Dancer 3D")',
 'Tum Se Hi (Reloaded) (From "Sadak 2")',
 'Bom Diggy Diggy',
 'Tere Siva',
 'Teri Bhabhi (From "Coolie No. 1")',
 'Husnn Hai Suhaana New (from "Coolie No. 1")',
 'MummyKassam (From "Coolie No. 1")',
 'Mirchi Lagi Toh (from "Coolie No. 1")',
 'Paani Paani',
 'Sawan Mein Lag Gayi Aag',
 'Nadiyon Paar (Let the Music Play Again) (From "Roohi")',
 'Lut Gaye',
 'Paani Paani',
 'Sawan Mein Lag Gayi Aag',
 'Teri Aankhon Mein']

In [67]:
song_df

,song_name,artist_name,played_at,timestamp
0,"Lakeeran (From ""Haseen Dillruba"")",Asees Kaur,2021-07-01T14:58:55.289Z,2021-07-01
1,"Tu Hi Yaar Mera (From ""Pati Patni Aur Woh"")",Rochak Kohli,2021-07-01T14:54:46.242Z,2021-07-01
2,"Illegal Weapon 2.0 (From ""Street Dancer 3D"")",Various Artists,2021-07-01T14:51:24.766Z,2021-07-01
3,"Lagdi Lahore Di (From ""Street Dancer 3D"")",Various Artists,2021-07-01T14:48:45.790Z,2021-07-01
4,"Tum Se Hi (Reloaded) (From ""Sadak 2"")",Ankit Tiwari,2021-07-01T14:38:54.861Z,2021-07-01
5,Bom Diggy Diggy,Zack Knight,2021-07-01T14:35:55.900Z,2021-07-01
6,Tere Siva,Javed-Mohsin,2021-07-01T14:32:25.225Z,2021-07-01
7,"Teri Bhabhi (From ""Coolie No. 1"")",Javed-Mohsin,2021-07-01T14:29:17.659Z,2021-07-01
8,"Husnn Hai Suhaana New (from ""Coolie No. 1"")",Various Artists,2021-07-01T14:25:25.772Z,2021-07-01
9,"MummyKassam (From ""Coolie No. 1"")",Various Artists,2021-07-01T14:20:57.892Z,2021-07-01
